In [6]:
import requests
from Bio import SeqIO
from Bio.KEGG import REST
from Bio.KEGG import Enzyme
%matplotlib inline
from IPython.display import Image
import io
import os
import pandas as pd
from Bio.KEGG.KGML import KGML_parser
from Bio.Graphics.KGML_vis import KGMLCanvas
from tqdm import tqdm
import json

In [3]:
def get_ncbi_protein_id(kegg_gene_id):
    base_url = "http://rest.kegg.jp"
    get_conv_url = f"{base_url}/conv/ncbi-proteinid/{kegg_gene_id}"
    #ncbi_protein_ids = []
    response = requests.get(get_conv_url)
    if response.status_code != 200:
        #print(f"Conversoin of Kegg ID to NCBI gene id failed (status code: {response.status_code})")
        return 'Nothing'
    for line in response.text.splitlines():
            #print(line)
            parts = line.split()
            #print(parts)
            if len(parts)==0:
                return 'Nothing'
            return parts[1]
            #print(parts[1])
            #ncbi_protein_ids.append(parts[1])

In [4]:
def get_uniprot_id(kegg_gene_id):
    base_url = "http://rest.kegg.jp"
    get_conv_url = f"{base_url}/conv/uniprot/{kegg_gene_id}"
    #ncbi_protein_ids = []
    response = requests.get(get_conv_url)
    if response.status_code != 200:
        #print(f"Conversoin of Kegg ID to NCBI gene id failed (status code: {response.status_code})")
        gene_id=kegg_gene_id.split(':')[1]
        get_conv_url = f"{base_url}/conv/uniprot/ncbi-proteinid:{kegg_gene_id}"
        response = requests.get(get_conv_url)
        if response.status_code != 200:
            return 'Nothing'
    for line in response.text.splitlines():
            #print(line)
            parts = line.split()
            #print(parts)
            if len(parts)==0:
                return 'Nothing'
            return parts[1]
            #print(parts[1])
            #ncbi_protein_ids.append(parts[1])
            

In [10]:
def get_protein_sequence(uniprot_id):
    """
    Fetches the amino acid sequence of a protein from UniProt.

    Args:
        uniprot_id (str): The UniProt ID of the protein.

    Returns:
        str: The amino acid sequence of the protein, or None if not found.
    """

    url = f'https://www.uniprot.org/uniprot/{uniprot_id}.fasta'
    response = requests.get(url)

    if response.status_code == 200:
        # Separate header from sequence
        header, sequence = response.text.split('\n', 1)
        return sequence.replace('\n', '')  # Remove newlines
    else:
        print(f"Protein not found (UniProt ID: {uniprot_id})")
        return None

# Example usage
#uniprot_id = "P01308"  # Example: Human Insulin
'''
uniprot_id = "G1JUH1"
sequence = get_protein_sequence(uniprot_id)

if sequence:
    print(sequence)
'''

'\nuniprot_id = "G1JUH1"\nsequence = get_protein_sequence(uniprot_id)\n\nif sequence:\n    print(sequence)\n'

In [27]:
def get_uniprot_ids_from_rhea_id(rhea_id):
    """
    Fetches a list of UniProt IDs associated with a Rhea reaction ID.

    Args:
        rhea_id: The Rhea reaction ID as a string (e.g., "RHEA:10000").

    Returns:
        A list of UniProt IDs, or None if an error occurs.
    """

    url= "https://rest.uniprot.org/uniprotkb/search?"
    rid = rhea_id.split(':')[1]

    parameter = {"query":'((cc_catalytic_activity:"rhea:"'+rid+') AND (fragment:false) AND (reviewed:true))',
        #"query":'((cc_catalytic_activity:"rhea:"'+rid+') AND (fragment:false) AND (reviewed:false))',
        #"query":'(cc_catalytic_activity:"rhea:"'+rid+')',
        "fields":"accession,rhea",
        "format":'tsv',
    }
    response = requests.get(url,params=parameter)
    #print(response.content)

   
    data = response.text.strip()

    # Check if the response has multiple lines (i.e., multiple UniProt IDs)
    if '\n' in data:
        uniprot_ids = data.splitlines()[1:]  # Skip the header line
    else:
        uniprot_ids = [data] if data else []

    return uniprot_ids


In [20]:
def get_gene_sequences_from_rhea_reaction(all_rhea_ids, rule_id):
    sequences = []
    all_uniprot_ids = []
    print("__________")
    for each_rhea_id in all_rhea_ids:
        uniprot_ids = get_uniprot_ids_from_rhea_id('rhea:'+each_rhea_id)
        print(each_rhea_id)
        print(uniprot_ids)
        if uniprot_ids:
            #print("UniProt IDs for", each_rhea_id)
            #print(len(uniprot_ids))
            for uniprot_id in uniprot_ids:
                p = ["uniprot_id:"+uniprot_id.split('\t')[0],get_protein_sequence(uniprot_id.split('\t')[0])]
                if uniprot_id.split('\t')[0]!='Entry' and uniprot_id.split('\t')[0] not in all_uniprot_ids:
                    #print(uniprot_id.split('\t')[0])
                    sequences.append(p)
                    #seq_record[each_rhea_id]=p
        #print("----------")
        #print(seq_record)
    return sequences
    

In [38]:
def get_gene_sequences_from_kegg_reaction(all_kegg_ids, rule_id):
    """
    Retrieves gene sequences associated with a KEGG reaction ID using the KEGG REST API.

    Args:
        reaction_id (str): The KEGG reaction ID (e.g., "R00001").

    Returns:
        list: A list of Biopython SeqRecord objects containing the gene sequences.
        
    """
    all_gene_ids_unique = []
    sequences = []
    all_ko_ids = []
    for reaction_id in all_kegg_ids:
        print(reaction_id)
        base_url = "http://rest.kegg.jp"

        # 1. Get EC number linked to the reaction
        get_ec_url = f"{base_url}/link/enzyme/{reaction_id}"
        response = requests.get(get_ec_url)
        print(get_ec_url)
        if response.status_code != 200:
            print(f"KEGG API request failed for EC number (status code: {response.status_code})")
            return []
            #raise ValueError(f"KEGG API request failed (status code: {response.status_code

        ec_numbers = []
        for line in response.text.splitlines():
            parts = line.split()
            ec_number = parts[1]  # EC number is in the second column
            ec_numbers.append(ec_number)

        # 2. Get KO number(Orthology number) linked to the EC number
        get_ko_url = f"{base_url}/link/ko/{ec_numbers[0]}"
        print(get_ko_url)
        response = requests.get(get_ko_url)

        if response.status_code != 200:
            print(f"KEGG API request failed for KO (status code: {response.status_code})")
            return []
        ko_ids = []
        
        for line in response.text.splitlines():
            parts = line.split()
            ko_id = parts[1].split(':')[1]
            if ko_id not in all_ko_ids:
                all_ko_ids.append(ko_id)

                # 3. For each KO number extracted, now extracting gene id for that KO number

                get_genes_url = f"{base_url}/link/genes/{ko_id}"
                print(get_genes_url)
                response = requests.get(get_genes_url)
                if response.status_code != 200:
                    print(f"KEGG API request failed for KO (status code: {response.status_code})")
                    return []

                gene_ids = []
                for line in response.text.splitlines():
                    parts = line.split()
                    gene_id = parts[1]

                    # 4. Adding only the genes which are not extracted before

                    if gene_id not in all_gene_ids_unique:
                        gene_ids.append(gene_id)
                        all_gene_ids_unique.append(gene_id)

                # 5. Fetch gene sequence data from each gene id

                for gene_id in gene_ids:
                    get_gene_url = f"{base_url}/get/{gene_id}/aaseq"
                    response = requests.get(get_gene_url)

                    if response.status_code != 200:
                        print(f"Warning: Failed to retrieve sequence for gene {gene_id}")
                        continue
                    itr = 0
                    seq_record = ''
                    for line in response.text.splitlines():
                        itr+=1
                        parts = line.split()
                        #print(itr)
                        if itr>1:
                            seq_record += parts[0]
                    record = ["kegg:"+gene_id.split(':')[0]]
                    record.append("ncbi_gene_id:"+gene_id.split(':')[1])
                    print(gene_id)
                    ncbi_protein_id = get_ncbi_protein_id(gene_id)
                    uniprot_id = get_uniprot_id(gene_id)
                    if ncbi_protein_id!='Nothing':
                        record.append("ncbi_protein_id:"+ncbi_protein_id.split(':')[1])
                    if uniprot_id!='Nothing':
                        record.append("uniprot_id:"+uniprot_id.split(':')[1])
                    print(record)
                    record.append(seq_record)
                    sequences.append(record)
                    #print(record)
                    '''
                    with open(f"{rule_id}_genes_{ko_ids[p]}.json", "w") as outfile: 
                        json.dump({reaction_id:sequences}, outfile)
                    '''
        
    
    return sequences

In [13]:
all_refs_in_all_rules = json.load(open('./all_refs_in_all_rules.json'))

In [14]:
all_refs_in_all_rules['MNXR102311']

{'kegg': ['R05552'],
 'rhea': [],
 'sabio': ['5157'],
 'metacyc': [],
 'seed': ['rxn03840'],
 'bigg': ['PABB', 'R_PABB']}

In [15]:
all_refs_in_all_rules['MNXR114039']

{'kegg': ['R10721'],
 'rhea': ['41432', '41433', '41434', '41435'],
 'sabio': ['12171', '6632', '6679'],
 'metacyc': ['RXN-15302'],
 'seed': ['rxn40455'],
 'bigg': []}

In [19]:
all_refs_in_all_rules['MNXR100024']

{'kegg': ['R00253', 'R00482', 'R02711', 'R05030', 'R00483'],
 'rhea': ['16169',
  '16170',
  '16171',
  '16172',
  '14197',
  '14198',
  '14199',
  '14200',
  '13853',
  '13854',
  '13855',
  '13856',
  '11372',
  '11373',
  '11374',
  '11375'],
 'sabio': ['760'],
 'metacyc': ['GLUTAMINESYN-RXN',
  '6.3.1.4-RXN',
  '4-METHYLENEGLUTAMATE--AMMONIA-LIGASE-RXN',
  'RXN-11338',
  'ASNSYNA-RXN'],
 'seed': ['rxn00187',
  'rxn19835',
  'rxn19836',
  'rxn30022',
  'rxn33952',
  'rxn33953',
  'rxn34791',
  'rxn34792',
  'rxn00339',
  'rxn44379',
  'rxn01956',
  'rxn46144',
  'rxn21649',
  'rxn03407',
  'rxn30313',
  'rxn00340',
  'rxn29993'],
 'bigg': ['GALh',
  'GALm',
  'GLNS',
  'GLNS_1',
  'R_GALh',
  'R_GALm',
  'R_GLNS',
  'R_GLNS_1',
  'ASNS3',
  'R_ASNS3',
  'ASNS2',
  'R_ASNS2']}

In [21]:
rule_id = "MNXR100024"

In [31]:
sequences_from_rhea = get_gene_sequences_from_rhea_reaction(all_refs_in_all_rules[rule_id]['rhea'], rule_id)

In [39]:
sequences_from_kegg = get_gene_sequences_from_kegg_reaction(all_refs_in_all_rules[rule_id]['kegg'], rule_id)

R00253
http://rest.kegg.jp/link/enzyme/R00253
http://rest.kegg.jp/link/ko/ec:6.3.1.2
http://rest.kegg.jp/link/genes/K01915
hsa:2752
['kegg:hsa', 'ncbi_gene_id:2752', 'ncbi_protein_id:NP_001028216', 'uniprot_id:P15104']
ptr:456450
['kegg:ptr', 'ncbi_gene_id:456450', 'ncbi_protein_id:NP_001266714', 'uniprot_id:G2HER6']
pps:100975375
['kegg:pps', 'ncbi_gene_id:100975375', 'ncbi_protein_id:XP_003824783', 'uniprot_id:A0A2R9CBD9']
ggo:101154019
['kegg:ggo', 'ncbi_gene_id:101154019', 'ncbi_protein_id:XP_004028058', 'uniprot_id:G3SJV7']
pon:100452390
['kegg:pon', 'ncbi_gene_id:100452390', 'ncbi_protein_id:XP_002809762', 'uniprot_id:A0A2J8V806']
nle:100602624
['kegg:nle', 'ncbi_gene_id:100602624', 'ncbi_protein_id:XP_030674647']
hmh:116458010
['kegg:hmh', 'ncbi_gene_id:116458010', 'ncbi_protein_id:XP_031993425']
mcc:715952
['kegg:mcc', 'ncbi_gene_id:715952', 'ncbi_protein_id:NP_001252942', 'uniprot_id:F7CAM5']
mcf:101926704
['kegg:mcf', 'ncbi_gene_id:101926704', 'ncbi_protein_id:XP_005540219', 

ConnectTimeout: HTTPSConnectionPool(host='rest.kegg.jp', port=443): Max retries exceeded with url: /conv/uniprot/vcm:VCM66_2666 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x12ec29bd0>, 'Connection to rest.kegg.jp timed out. (connect timeout=None)'))

In [ ]:
 with open("MNXR100024_genes_kegg.json", "w") as outfile:
        json.dump({"MNXR100024":sequences_from_kegg}, outfile)